In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [2]:
dataset=pd.read_csv(r"C:\Users\YAMINI RAVICHANDRAN\OneDrive\ドキュメント\Flight Fare\Flight_Price.csv")
dataset.head()


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [4]:
dataset.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [5]:
dataset.dropna(inplace=True)

In [6]:
dataset.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [7]:
dataset['Journey_day']=pd.to_datetime(dataset['Date_of_Journey'],format='%d/%m/%Y').dt.day
dataset['Journey_month']=pd.to_datetime(dataset['Date_of_Journey'],format='%d/%m/%Y').dt.month
dataset.drop(['Date_of_Journey'],axis=1,inplace=True)

dataset['dep_hour']=pd.to_datetime(dataset['Dep_Time']).dt.hour
dataset['dep_min']=pd.to_datetime(dataset['Dep_Time']).dt.minute
dataset.drop(['Dep_Time'],axis=1,inplace=True)

dataset['arrival_hour']=pd.to_datetime(dataset['Arrival_Time']).dt.hour
dataset['arrival_min']=pd.to_datetime(dataset['Arrival_Time']).dt.minute
dataset.drop(['Arrival_Time'],axis=1,inplace=True)

duration = list(dataset['Duration'])

for i in range(len(duration)):
    duration[i] = duration[i].strip()  
    if len(duration[i].split()) != 2: 
        if "h" in duration[i]:
            duration[i] = duration[i] + " 0m" 
        else:
            duration[i] = "0h " + duration[i]  


duration_hours = []
duration_min = []

for i in range(len(duration)):
    if "h" in duration[i]:
        hours = int(duration[i].split(sep="h")[0])
    else:
        hours = 0

    
    if "m" in duration[i]:
        minutes = int(duration[i].split(sep="m")[0].split()[-1])
    else:
        minutes = 0

    duration_hours.append(hours)
    duration_min.append(minutes)


dataset['Duration_hours'] = duration_hours
dataset['Duration_mins'] = duration_min
dataset.drop(['Duration'], axis=1, inplace=True)


dataset.head()

C:\Users\YAMINI RAVICHANDRAN\AppData\Local\Temp\ipykernel_13252\1494020137.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['dep_hour']=pd.to_datetime(dataset['Dep_Time']).dt.hour
C:\Users\YAMINI RAVICHANDRAN\AppData\Local\Temp\ipykernel_13252\1494020137.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['dep_min']=pd.to_datetime(dataset['Dep_Time']).dt.minute
C:\Users\YAMINI RAVICHANDRAN\AppData\Local\Temp\ipykernel_13252\1494020137.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset['arrival_hour']=pd.to_datetime(dataset['Arrival_Time']).dt.hou

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey_day,Journey_month,dep_hour,dep_min,arrival_hour,arrival_min,Duration_hours,Duration_mins
0,IndiGo,Banglore,New Delhi,BLR ? DEL,non-stop,No info,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,2 stops,No info,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL ? LKO ? BOM ? COK,2 stops,No info,13882,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,CCU ? NAG ? BLR,1 stop,No info,6218,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,BLR ? NAG ? DEL,1 stop,No info,13302,1,3,16,50,21,35,4,45


In [8]:
dataset['Airline'].value_counts()

Airline
Jet Airways                          3849
IndiGo                               2053
Air India                            1751
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: count, dtype: int64

In [9]:
Airline = dataset[["Airline"]]
Current_Airline_List = Airline['Airline']
New_Airline_List = []

for carrier in Current_Airline_List:
  if carrier in ['Jet Airways', 'IndiGo', 'Air India', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia']:
    New_Airline_List.append(carrier)
  else:
    New_Airline_List.append('Other')

Airline['Airline'] = pd.DataFrame(New_Airline_List)
Airline['Airline'].value_counts()

C:\Users\YAMINI RAVICHANDRAN\AppData\Local\Temp\ipykernel_13252\737743019.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Airline['Airline'] = pd.DataFrame(New_Airline_List)


Airline
Jet Airways          3849
IndiGo               2053
Air India            1750
Multiple carriers    1196
SpiceJet              818
Vistara               479
Air Asia              319
GoAir                 194
Other                  23
Name: count, dtype: int64

In [10]:
dataset=pd.get_dummies(dataset,columns=['Airline'],dtype=int,drop_first=True)
dataset

,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey_day,Journey_month,dep_hour,dep_min,...,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy
0,Banglore,New Delhi,BLR ? DEL,non-stop,No info,3897,24,3,22,20,...,0,1,0,0,0,0,0,0,0,0
1,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,2 stops,No info,7662,1,5,5,50,...,0,0,0,0,0,0,0,0,0,0
2,Delhi,Cochin,DEL ? LKO ? BOM ? COK,2 stops,No info,13882,9,6,9,25,...,0,0,1,0,0,0,0,0,0,0
3,Kolkata,Banglore,CCU ? NAG ? BLR,1 stop,No info,6218,12,5,18,5,...,0,1,0,0,0,0,0,0,0,0
4,Banglore,New Delhi,BLR ? NAG ? DEL,1 stop,No info,13302,1,3,16,50,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Kolkata,Banglore,CCU ? BLR,non-stop,No info,4107,9,4,19,55,...,0,0,0,0,0,0,0,0,0,0
10679,Kolkata,Banglore,CCU ? BLR,non-stop,No info,4145,27,4,20,45,...,0,0,0,0,0,0,0,0,0,0
10680,Banglore,Delhi,BLR ? DEL,non-stop,No info,7229,27,4,8,20,...,0,0,1,0,0,0,0,0,0,0
10681,Banglore,New Delhi,BLR ? DEL,non-stop,No info,12648,1,3,11,30,...,0,0,0,0,0,0,0,0,1,0


In [11]:
dataset['Source'].value_counts()

Source
Delhi       4536
Kolkata     2871
Banglore    2197
Mumbai       697
Chennai      381
Name: count, dtype: int64

In [13]:
Source = dataset[["Source"]]
Source = pd.get_dummies(Source,dtype=int, drop_first= True) 
Source.head()

,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai
0,0,0,0,0
1,0,0,1,0
2,0,1,0,0
3,0,0,1,0
4,0,0,0,0


In [14]:
dataset['Destination'].value_counts()

Destination
Cochin       4536
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kolkata       381
Name: count, dtype: int64

In [29]:
Current_Destination_List = dataset["Destination"]

Current_Destination_List=Current_Destination_List.replace("New Delhi","Delhi")
Current_Destination_List.head()

Destination=pd.get_dummies(Current_Destination_List,dtype=int)
Destination



,Banglore,Cochin,Delhi,Hyderabad,Kolkata
0,0,0,1,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,1,0,0,0,0
4,0,0,1,0,0
...,...,...,...,...,...
10678,1,0,0,0,0
10679,1,0,0,0,0
10680,0,0,1,0,0
10681,0,0,1,0,0


In [30]:
dataset.drop(["Route", "Additional_Info"], axis = 1, inplace = True)

KeyError: "['Route', 'Additional_Info'] not found in axis"

In [31]:
dataset["Total_Stops"].value_counts()

Total_Stops
1    5625
0    3491
2    1520
3      45
4       1
Name: count, dtype: int64

In [32]:
dataset.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)
dataset.head()

,Source,Destination,Total_Stops,Price,Journey_day,Journey_month,dep_hour,dep_min,arrival_hour,arrival_min,...,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy
0,Banglore,Delhi,0,3897,24,3,22,20,1,10,...,0,1,0,0,0,0,0,0,0,0
1,Kolkata,Banglore,2,7662,1,5,5,50,13,15,...,0,0,0,0,0,0,0,0,0,0
2,Delhi,Cochin,2,13882,9,6,9,25,4,25,...,0,0,1,0,0,0,0,0,0,0
3,Kolkata,Banglore,1,6218,12,5,18,5,23,30,...,0,1,0,0,0,0,0,0,0,0
4,Banglore,Delhi,1,13302,1,3,16,50,21,35,...,0,1,0,0,0,0,0,0,0,0


In [33]:
data_train = pd.concat([dataset, Airline, Source, Destination], axis = 1) # axis = 1 signifies column
data_train.drop(["Airline", "Source", "Destination"], axis = 1, inplace = True)

data_train.head()

,Total_Stops,Price,Journey_day,Journey_month,dep_hour,dep_min,arrival_hour,arrival_min,Duration_hours,Duration_mins,...,Airline_Vistara Premium economy,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Banglore,Cochin,Delhi,Hyderabad,Kolkata
0,0,3897,24,3,22,20,1,10,2,50,...,0,0,0,0,0,0,0,1,0,0
1,2,7662,1,5,5,50,13,15,7,25,...,0,0,0,1,0,1,0,0,0,0
2,2,13882,9,6,9,25,4,25,19,0,...,0,0,1,0,0,0,1,0,0,0
3,1,6218,12,5,18,5,23,30,5,25,...,0,0,0,1,0,1,0,0,0,0
4,1,13302,1,3,16,50,21,35,4,45,...,0,0,0,0,0,0,0,1,0,0


In [34]:
data_train.to_csv('data_train.csv', index=False)


In [35]:
data_train.shape


(10682, 30)

In [36]:
X=data_train.drop(['Price'],axis=True)
y=data_train['Price']
print(X.shape, y.shape)

(10682, 29) (10682,)


In [3]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Ensure the experiment is set correctly
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Flight Fare Prediction")

# Assuming your data is in the 'data_train' variable
df = pd.read_csv(r"C:\Users\YAMINI RAVICHANDRAN\OneDrive\ドキュメント\Flight Fare\data_train.csv")
X = df.drop(['Price'], axis=1)
y = df['Price']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# List of models to evaluate
models = [LinearRegression(), Lasso(), Ridge(), RandomForestRegressor(), GradientBoostingRegressor()]

# Start an MLflow run for each model
for model in models:
    with mlflow.start_run():
        # Log experiment info (print current experiment ID)
        print(f"Logging to experiment: {mlflow.active_run().info.experiment_id}")

        # Train the model
        model.fit(x_train, y_train)

        # Predictions
        train_prediction = model.predict(x_train)
        test_prediction = model.predict(x_test)
        
        # Metrics
        mse_train = mean_squared_error(y_train, train_prediction)
        mae_train = mean_absolute_error(y_train, train_prediction)
        r2_train = r2_score(y_train, train_prediction)
        
        mse_test = mean_squared_error(y_test, test_prediction)
        mae_test = mean_absolute_error(y_test, test_prediction)
        r2_test = r2_score(y_test, test_prediction)

        # Log model parameters and metrics
        mlflow.log_param("model_type", type(model).__name__)
        mlflow.log_param("test_size", 0.2)
        
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("r2_train", r2_train)
        
        mlflow.log_metric("mse_test", mse_test)
        mlflow.log_metric("mae_test", mae_test)
        mlflow.log_metric("r2_test", r2_test)
        
        # Log the trained model as an artifact
        mlflow.sklearn.log_model(model, artifact_path=f"{type(model).__name__}_model")

        # Print results for monitoring
        print(f"{type(model).__name__}")
        print("****train****")
        print(f"mse train: {mse_train}")
        print(f"mae train: {mae_train}")
        print(f"r2 train: {r2_train}")
        print("****test****")
        print(f"mse test: {mse_test}")
        print(f"mae test: {mae_test}")
        print(f"r2 test: {r2_test}")
        print("\n")


Logging to experiment: 788635645193710992


2025/01/23 10:23:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LinearRegression
****train****
mse train: 8574510.87040659
mae train: 1996.9179050803943
r2 train: 0.6103178114893199
****test****
mse test: 7054144.209593428
mae test: 1911.086115254026
r2 test: 0.6144966116867681


🏃 View run languid-ray-986 at: http://127.0.0.1:5000/#/experiments/788635645193710992/runs/d8172965e2a24f3db133a434ac0cd5b2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/788635645193710992
Logging to experiment: 788635645193710992


2025/01/23 10:23:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Lasso
****train****
mse train: 8581905.511192152
mae train: 1997.8669782035658
r2 train: 0.6099817503602223
****test****
mse test: 7057894.506969668
mae test: 1912.8033800095848
r2 test: 0.614291660908507


🏃 View run whimsical-rook-338 at: http://127.0.0.1:5000/#/experiments/788635645193710992/runs/7f1ba5cc38284425afaa793950274d09
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/788635645193710992
Logging to experiment: 788635645193710992


...

Ridge
****train****
mse train: 8609312.681577291
mae train: 1999.9661845304975
r2 train: 0.6087361882170303
****test****
mse test: 7062334.97082647
mae test: 1914.6410413568772
r2 test: 0.6140489930792681


🏃 View run chill-carp-593 at: http://127.0.0.1:5000/#/experiments/788635645193710992/runs/cabbd872469d4a9e905d55c1a61e8c3e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/788635645193710992
Logging to experiment: 788635645193710992


2025/01/23 10:23:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForestRegressor
****train****
mse train: 1000346.873550418
mae train: 564.3474889487601
r2 train: 0.9545376564510135
****test****
mse test: 3249023.7961264304
mae test: 1154.4636192590751
r2 test: 0.822443425467021


🏃 View run rebellious-crow-364 at: http://127.0.0.1:5000/#/experiments/788635645193710992/runs/b70e74472a12490192867bc3d1249753
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/788635645193710992
Logging to experiment: 788635645193710992


2025/01/23 10:23:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


GradientBoostingRegressor
****train****
mse train: 4746322.585027958
mae train: 1501.0410663104108
r2 train: 0.7842958741011357
****test****
mse test: 4140953.522989214
mae test: 1521.9884519685213
r2 test: 0.7737001730431077


🏃 View run bittersweet-ram-823 at: http://127.0.0.1:5000/#/experiments/788635645193710992/runs/0ff8fbea916d44b6af47786d07dda4e6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/788635645193710992


In [40]:
modelname="RandomForestRegressor"
run_id="f79b60c3b3b0464f8a02ad81eeabc53d"
model_uri=f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri,name=modelname)


Successfully registered model 'RandomForestRegressor'.
2025/01/20 11:50:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestRegressor, version 1


🏃 View run RandomForestRegressor at: http://127.0.0.1:5000/#/experiments/313017008437592976/runs/f79b60c3b3b0464f8a02ad81eeabc53d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/313017008437592976


Created version '1' of model 'RandomForestRegressor'.


In [2]:
import pickle


In [8]:
import pandas as pd

In [11]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
df = pd.read_csv(r"C:\Users\YAMINI RAVICHANDRAN\OneDrive\ドキュメント\Flight Fare\data_train.csv")
X = df.drop(['Price'], axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

# Save the model
with open("flight_fare_model.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model saved as flight_fare_model.pkl")


MAE: 1183.1904935112332
MSE: 4443090.327137265
Model saved as flight_fare_model.pkl


In [13]:
training_columns = X_train.columns.tolist()  # X_train is the training dataset with features
# Save model and columns during training
with open('flight_fare_model.pkl', 'wb') as file:
    pickle.dump((model, training_columns), file)
